In [ ]:
import numpy as np
import torch
import data_making.dataset as dataset
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import results
import os
import trimesh
from mesh_to_sdf import sample_sdf_near_surface
import data
from utils import utils
import meshplot as mp
from data_making.extract_data import *
import argparse
from mesh_to_sdf.surface_point_cloud import *

### PartNet Mobility extracted with my functions

In [ ]:
data_set = dataset.SDFDataset()

In [ ]:
sdf = np.array(data_set[7000:14000][1].view(-1))
plt.hist(sdf, bins=40)
plt.show()

In [ ]:
obj_ids = torch.where(data_set[:][0][:, 0] == 3763.0)[0]
obj_ids

In [ ]:
obj_ids = torch.unique(data_set[:][0][:, 0])

# CHANGE HERE TO CHANGE OBJECT ----------------
obj_id = obj_ids[2]
# CHANGE HERE TO CHANGE OBJECT ----------------

obj_ixs = torch.where(data_set[:][0][:, 0] == obj_id)[0]
coords_latent = data_set[obj_ixs][0]
points = coords_latent[:, 1:]
sdf = data_set[obj_ixs][1]

In [ ]:
np.where(sdf < 0)[0]

In [ ]:
negative_indices = np.where(sdf < 0)[0]
positive_indices = np.where(sdf > 0)[0]

negative_sdf_points = points[negative_indices]
positive_sdf_points = points[positive_indices]

print(f'Number of positive coordinates: {positive_sdf_points.shape[0]}')
print(f'Number of negative coordinates: {negative_sdf_points.shape[0]}')

fig = go.Figure(
    [
        go.Scatter3d(x=negative_sdf_points[:, 0], 
             y=negative_sdf_points[:, 1],
             z=negative_sdf_points[:, 2], 
             mode='markers', marker=dict(size=2)),
        go.Scatter3d(x=positive_sdf_points[:, 0], 
             y=positive_sdf_points[:, 1],
             z=positive_sdf_points[:, 2], 
             mode='markers', marker=dict(size=2, color='orange')
            )
    ]
)
fig.show()

The problem is that only a tiny fraction of the overall points is associated with a negative sdf. A possible solution is to sample more points near the negative samples and only keep the negative ones.

In [ ]:
negative_sdf_points.shape

In [ ]:
objs_dict = np.load(os.path.join(os.path.dirname(results.__file__), 'objs_dict.npy'), allow_pickle=True).item()
objs_dict.keys()

In [ ]:
objs_dict = np.load(os.path.join(os.path.dirname(results.__file__), 'objs_dict.npy'), allow_pickle=True).item()
obj_idx = '102505'
mesh = trimesh.Trimesh(objs_dict[obj_idx]['verts'], objs_dict[obj_idx]['faces'])


In [ ]:
sigma = 0.01
spc = create_from_scans(mesh)

for i in range(4):
    additional_negatives = np.random.normal(0, sigma, size=(negative_sdf_points.shape[0] * negative_sdf_points.shape[1]))
    temp_negative_sdf_points = negative_sdf_points + additional_negatives.reshape(negative_sdf_points.shape[0], negative_sdf_points.shape[1]) 
    print(temp_negative_sdf_points)
    sdf = spc.get_sdf_in_batches(temp_negative_sdf_points, use_depth_buffer=True)
    new_neg_points = temp_negative_sdf_points[sdf<0]
    negative_sdf_points = np.vstack((negative_sdf_points, new_neg_points))


In [ ]:
# doppia only points at the surface
coords_latent = data_set[50000:100000][0]
points = coords_latent[:, 1:]
sdf = data_set[50000:100000][1]

negative_indices = np.where(sdf < 0)[0]
positive_indices = np.where(sdf > 0)[0]

negative_sdf_points = points[negative_indices]
positive_sdf_points = points[positive_indices]

print(f'Number of positive coordinates: {positive_sdf_points.shape[0]}')
print(f'Number of negative coordinates: {negative_sdf_points.shape[0]}')

surface_indices = np.where((sdf > -0.01) & (sdf < 0.01))[0]
points_on_surface = points[surface_indices]

fig = go.Figure(
    [
        go.Scatter3d(x=points_on_surface[:, 0], 
                     y=points_on_surface[:, 1],
                     z=points_on_surface[:, 2], 
                     mode='markers', marker=dict(size=2))
    ]
)
fig.show()
sdf_on_surface = sdf[surface_indices]

objs_dict = np.load(os.path.join(os.path.dirname(results.__file__), 'objs_dict.npy'), allow_pickle=True).item()
obj_idx = '102505'
mesh = trimesh.Trimesh(objs_dict[obj_idx]['verts'], objs_dict[obj_idx]['faces'])
spc = create_from_scans(mesh)
sigma = 0.05

for _ in range(2):
    additional_points = torch.normal(0, sigma, size=(points_on_surface.shape[0] * points_on_surface.shape[1], ))
    temp_points = points_on_surface + additional_points.view(points_on_surface.shape[0], points_on_surface.shape[1]) 
    temp_sdf = torch.tensor(spc.get_sdf_in_batches(temp_points, use_depth_buffer=True))
    new_neg_points = temp_points[temp_sdf<0]
    new_sdf = temp_sdf[temp_sdf < 0].view(-1, 1)
    sdf = np.vstack((sdf, new_sdf))
    points = torch.vstack((points, new_neg_points))
    surface_indices = np.where((sdf > -0.01) & (sdf < 0.01))[0]
    points_on_surface = points[surface_indices]
    
negative_indices = np.where(sdf < 0)[0]
positive_indices = np.where(sdf > 0)[0]

negative_sdf_points = points[negative_indices]
positive_sdf_points = points[positive_indices]

print(f'Number of new positive coordinates: {positive_sdf_points.shape[0]}')
print(f'Number of new negative coordinates: {negative_sdf_points.shape[0]}')

fig = go.Figure(
    [
        go.Scatter3d(x=negative_sdf_points[:, 0], 
                     y=negative_sdf_points[:, 1],
                     z=negative_sdf_points[:, 2], 
                     mode='markers', marker=dict(size=2)),
        go.Scatter3d(x=positive_sdf_points[:, 0], 
                     y=positive_sdf_points[:, 1],
                     z=positive_sdf_points[:, 2], 
                     mode='markers', marker=dict(size=2, color='orange'))
    ]
)
fig.show()

### PartNet Mobility with Marian's function

In [ ]:
objs_dict = np.load(os.path.join(os.path.dirname(results.__file__), 'objs_dict.npy'), allow_pickle=True).item()

In [ ]:
obj = objs_dict['102763']
verts = obj['verts']
faces = obj['faces']

In [ ]:
mesh = trimesh.Trimesh(verts, faces)
points, sdf = sample_sdf_near_surface(mesh, number_of_points=50000, sign_method='depth')

In [ ]:
plt.hist(sdf, bins=40)
plt.show()

In [ ]:
negative_indices = np.where(sdf < 0)
positive_indices = np.where(sdf > 0.001)

negative_sdf_points = points[negative_indices]
positive_sdf_points = points[positive_indices]

In [ ]:
fig = go.Figure(
    [
        go.Scatter3d(x=negative_sdf_points[:, 0], 
             y=negative_sdf_points[:, 1],
             z=negative_sdf_points[:, 2], 
             mode='markers', marker=dict(size=2)),
        go.Scatter3d(x=positive_sdf_points[:, 0], 
             y=positive_sdf_points[:, 1],
             z=positive_sdf_points[:, 2], 
             mode='markers', marker=dict(size=2, color='orange')
            )
    ]
)
fig.show()

### ShapeNet with my function

In [ ]:
obj_path = os.path.join(os.path.dirname(data.__file__), 'shapenet_obj', '1b29de0b8be4b18733d25da891be74b8.obj' )
mesh = utils._as_mesh(trimesh.load(obj_path))   # for some reason, they are 2

In [ ]:
mp.plot(mesh.vertices, mesh.faces)

In [ ]:
obj_idx = 0
#trimesh.repair.fix_inversion(mesh)
objs_dict = {0: {'verts': np.asarray(mesh.vertices), 'faces': np.asarray(mesh.faces)}}

class Args:
  num_samples_on_surface = 5000
  num_samples_far_surface = 5000

args=Args()

samples_dict = dict()
samples_dict[obj_idx] = dict()
samples_dict[obj_idx]['samples'] = sample(objs_dict[obj_idx], args)
samples_dict[obj_idx]['sdf'] = compute_sdf(objs_dict[obj_idx]['verts'], objs_dict[obj_idx]['faces'], samples_dict[obj_idx]['samples'])
samples_dict[obj_idx]['latent_class'] = np.array([obj_idx], dtype=np.int32)
samples_dict[obj_idx]['samples_latent_class'] = combine_sample_latent(samples_dict[obj_idx]['samples'], samples_dict[obj_idx]['latent_class'])


In [ ]:
def _debug_plot(samples, dist=True):
    points = samples['samples']
    sdfs = samples['sdf']
    inner = points[sdfs>0]
    outer = points[sdfs<0]

    fig = go.Figure(
        [
            go.Scatter3d(x=inner[:, 0], y=inner[:, 1],z=inner[:, 2], mode='markers', marker=dict(size=2)),
            go.Scatter3d(x=outer[:, 0], y=outer[:, 1],z=outer[:, 2], mode='markers', marker=dict(size=2))
        ]
    )
    fig.show()

_debug_plot(samples_dict[obj_idx]) 


In [ ]:
points, sdf = sample_sdf_near_surface(mesh, sign_method='depth', number_of_points=100000)

In [ ]:
plt.hist(sdf, bins=40)
plt.show()

In [ ]:
negative_indices = np.where(sdf < 0)
positive_indices = np.where(sdf > 0)

negative_sdf_points = points[negative_indices]
positive_sdf_points = points[positive_indices]

negative_sdf = sdf[negative_indices]
positive_sdf = sdf[positive_indices]

In [ ]:
fig = go.Figure(
    [
        go.Scatter3d(x=negative_sdf_points[:, 0], 
             y=negative_sdf_points[:, 1],
             z=negative_sdf_points[:, 2], 
             mode='markers', marker=dict(size=2, color='orange')),
        go.Scatter3d(x=positive_sdf_points[:, 0], 
             y=positive_sdf_points[:, 1],
             z=positive_sdf_points[:, 2], 
             mode='markers', marker=dict(size=2, color='blue')
            )
    ]
)
fig.show()

In [ ]:
np.amin(sdf)

In [ ]:
a = np.array([1, 5, 11])
np.where(a < 6)[0]

# Test new mesh_to_sdf method

In [1]:
import results
import numpy as np
import os
import trimesh
from mesh_to_sdf import sample_sdf_near_surface
import plotly.graph_objects as go


In [2]:
objs_dict = np.load(os.path.join(os.path.dirname(results.__file__), 'objs_dict.npy'), allow_pickle=True).item()
samples_dict = dict()
scatters = []
number_of_points=5000

for obj_idx in list(objs_dict.keys())[0:2]:
    samples_dict[obj_idx] = dict()
    mesh = trimesh.Trimesh(objs_dict[obj_idx]['verts'], objs_dict[obj_idx]['faces'])
    
    points, sdf = sample_sdf_near_surface(mesh, number_of_points=number_of_points, sign_method='depth')
    scatters.append(go.Scatter3d(
                     x=points[:, 0], 
                     y=points[:, 1],
                     z=points[:, 2], 
                     mode='markers', marker=dict(size=2, color='darkblue'),
                     name='unit_sphere'
                ))
    
    points, sdf = sample_sdf_near_surface(mesh, number_of_points=number_of_points, sign_method='depth', unit_sphere_scaler=False)
    scatters.append(go.Scatter3d(
                     x=points[:, 0], 
                     y=points[:, 1],
                     z=points[:, 2], 
                     mode='markers', marker=dict(size=2, color='lightblue'),
                     name='none'
                ))
    
    points, sdf = sample_sdf_near_surface(mesh, number_of_points=number_of_points, sign_method='depth', unit_cube_scaler=True)
    scatters.append(go.Scatter3d(
                     x=points[:, 0], 
                     y=points[:, 1],
                     z=points[:, 2], 
                     mode='markers', marker=dict(size=2, color='orange'),
                     name='unit_cube'
                ))
    
    fig = go.Figure(scatters)
    fig.show()

TypeError: sample_sdf_near_surface() got an unexpected keyword argument 'unit_sphere_scaler'

In [8]:
mesh.vertices

TrackedArray([[-0.00328553,  0.1601998 ,  0.70167664],
              [-0.00328553,  0.1601998 ,  0.992553  ],
              [ 0.16019925,  0.003285  ,  0.70167664],
              ...,
              [-0.13086742, -0.00268368,  0.68678362],
              [-0.13086742, -0.00268368,  0.72331809],
              [-0.13086742, -0.00268368,  0.67968587]])

In [7]:
np.linalg.norm(mesh.vertices, axis=1)

array([0.71973945, 1.00540352, 0.71973933, 1.00540343, 0.71973931,
       1.00540342, 0.71973933, 1.00540343, 0.7197395 , 1.00540356,
       0.71973953, 1.00540357, 0.71973944, 1.00540351, 0.71973957,
       1.0054036 , 1.01225381, 1.01225406, 1.01225396, 1.01225389,
       1.01225387, 1.01225389, 0.71973956, 1.0054036 , 0.71973934,
       1.00540344, 1.01225395, 1.01225384, 1.01225379, 0.71973936,
       1.00540346, 1.01225378, 0.71973947, 1.00540354, 0.71973935,
       1.00540345, 1.01225396, 1.01225384, 0.71973943, 1.00540351,
       0.71973942, 1.0054035 , 1.01225392, 1.01225379, 0.71973928,
       1.0054034 , 0.71973936, 1.00540346, 1.01225395, 1.01225394,
       0.7197394 , 1.00540349, 0.71973924, 1.00540337, 1.01225384,
       1.01225399, 1.01225389, 0.7197394 , 1.00540349, 0.7197395 ,
       1.00540355, 0.71973951, 1.00540356, 1.01225395, 1.01225388,
       1.01225383, 0.71973942, 1.0054035 , 0.71973943, 1.00540351,
       1.01225384, 1.01225395, 0.53980956, 0.53980941, 0.94378